In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

first_year = 1985
last_year = 2020

data_folder = "projects/amazon-forest-regrowth/assets"




Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [4]:
# This collection is not publicly accessible. To sign up for access,
# please see https://developers.planet.com/docs/integrations/gee/nicfi
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')

# # Filter basemaps by date and get the first image from filtered results
# basemap = nicfi.filter(ee.Filter.date('2016-03-01', '2016-07-01')).first()
# basemap2 = nicfi.filter(ee.Filter.date('2018-03-01', '2018-07-01')).first()
basemap = nicfi.filter(ee.Filter.date('2023-01-01', '2023-07-01')).first()

vis_planet = {'bands': ['R', 'G', 'B'], 'min': 64, 'max': 5454, 'gamma': 1.8}



In [6]:
roi = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filterMetadata("country_na", "equals", "Panama").geometry()

# Load the image collections
transition = ee.ImageCollection('projects/JRC/TMF/v1_2023/TransitionMap_Subtypes').mosaic().clip(roi)
annual_changes = ee.ImageCollection('projects/JRC/TMF/v1_2023/AnnualChanges').mosaic().clip(roi)

# Define regrowth and degraded conditions
regrowth = transition.gte(31).And(transition.lte(33))

# Initialize AgeRegrowth and AgeDegraded
age = ee.Image.constant(0)

# Calculate AgeRegrowth
for i in range(1990, 2023):
    year = 'Dec' + str(i)
    annual_changes_year = annual_changes.select(year)
    condition = annual_changes_year.eq(4).And(regrowth) # were regrowing then AND are regrowing now
    age = age.add(condition.eq(1))

age = age.selfMask().rename("age")

# Visualization parameters
vis_ages = {'min': 0, 'max': 35, 'palette': ['yellow', 'red']}

Map = geemap.Map()
Map.centerObject(basemap, 4)
Map.addLayer(basemap, vis_planet, '2023 mosaic')
Map.addLayer(age, vis_ages, 'ages')
Map

Map(center=[-0.0013335310118573354, -73.54611700899127], controls=(WidgetControl(options=['position', 'transpa…

In [9]:
mature = ee.Image(f"{data_folder}/mature_biomass")
agbd = ee.Image(f"{data_folder}/age_agbd").select('agbd')
mask = ee.Image(f"{data_folder}/mapbiomas/one_hectare_mask")
ages = ee.Image(f"{data_folder}/mapbiomas/secondary").select('age').updateMask(mask)
pantanal = ee.FeatureCollection(f"{data_folder}/raw/biomes_br") \
        .filter(ee.Filter.eq("CD_Bioma", 6)).geometry()

# Visualization parameters
vis_ages = {'min': 0, 'max': 35, 'palette': ['yellow', 'red']}
vis_agbd = {'min': 0, 'max': 300, 'palette': ['yellow', 'green']}
vis_mature = {'min': 0, 'max': 300, 'palette': ['blue', 'red']}
vis_geom = {
    'color': 'red',      # Color of the edge
    'width': 2           # Width of the edge
}

Map = geemap.Map()
Map.centerObject(basemap, 4)
# Map.addLayer(basemap, vis, '2016 mosaic')
Map.addLayer(basemap, vis_planet, '2020 mosaic')
# Map.addLayer(pantanal, vis_geom, 'pantanal')
Map.addLayer(ages, vis_ages, 'ages')
# Map.addLayer(lulc, {}, 'lulc')
# Map.addLayer(agbd, vis_agbd, 'agbd')
# Map.addLayer(mature, vis_mature, 'mature')
Map

Map(center=[-0.03247460684437231, -73.82115720575285], controls=(WidgetControl(options=['position', 'transpare…